In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import cv2
from IPython.display import clear_output, display
import ipywidgets as widgets
import PIL.Image
from io import BytesIO
import numpy as np

import torch
from models import Darknet
from utils.utils import non_max_suppression
from utils import *

classes = utils.load_classes('config/coco.names')
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [26]:
np.mod(4, skip)

0

In [ ]:
cuda = False
# Set up model
model = Darknet('config/yolov3.cfg', img_size=416)
model.load_weights('weights/yolov3.weights')

if cuda:
    model.cuda()

model.eval() # Set in evaluation mode

videopath = 'Amsterdam_drive_small_crop.mp4'
vid = cv2.VideoCapture(videopath)
out = widgets.Image(layout=widgets.Layout(height='360px', width='360px'))
display(out)

skip, nframe = 2, -1
try:
    while True:
        # Get and prepare frame
        ret, frame = vid.read()
        if not ret:
            break
        nframe += 1
        if not np.mod(nframe, skip) == 0:
            continue
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)        
        input_imgs = torch.from_numpy(frame/255.).unsqueeze(0).type('torch.FloatTensor')
        input_imgs = torch.transpose(input_imgs, 1, 3)
        input_imgs = torch.transpose(input_imgs, 2, 3)
        if cuda: 
            input_imgs = input_imgs.cuda()
        # Get detections
        with torch.no_grad():
            detections = model(input_imgs)
            detections = non_max_suppression(detections, len(classes), 0.7, 0.4)
        detections = detections[0]
         # Draw bounding boxes and labels of detections
        if detections is not None:
            for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
                # Draw detections on the frame
                color = colors[int(cls_pred)]
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(frame, classes[int(cls_pred)], 
                            (int(x1)-10, int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        # Update image widget
        f = BytesIO() 
        PIL.Image.fromarray(frame).save(f, 'JPEG')
        out.value = f.getvalue()
        #cv2.waitKey(10)
        
except KeyboardInterrupt:
    vid.release()    

/usr/local/lib/python3.5/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Image(value=b'', layout="Layout(height='360px', width='360px')")

/usr/local/lib/python3.5/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
